# Urban Growth Prediction in Trentino - Quick Start Example

This notebook demonstrates the basic usage of the urban growth prediction framework.

## 1. Setup and Configuration

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path
sys.path.insert(0, os.path.join(os.getcwd(), '../src'))

from src.utils import load_config
from src.signal_processing import SpatioTemporalProcessor
from src.models import UrbanGrowthPredictor, create_synthetic_dataset
from src.utils.visualization import plot_confusion_matrix, plot_feature_importance

## 2. Signal Processing Example

In [ ]:
# Create a processor
processor = SpatioTemporalProcessor(tile_size=1000, overlap=100)

# Example time series (simulating NDBI over 5 years)
time_series = np.array([0.1, 0.12, 0.15, 0.18, 0.22, 0.25])
timestamps = np.array([0, 1, 2, 3, 4, 5])

# Detect trend
trend = processor.detect_trend(time_series, timestamps, method='linear_regression')
print("Trend Analysis:")
print(f"  Slope: {trend['slope']:.4f}")
print(f"  R²: {trend['r_squared']:.4f}")
print(f"  P-value: {trend['p_value']:.4f}")

In [ ]:
# Visualize trend
plt.figure(figsize=(10, 6))
plt.plot(timestamps, time_series, 'o-', label='Data', linewidth=2)
trend_line = trend['slope'] * timestamps + trend['intercept']
plt.plot(timestamps, trend_line, 'r--', label=f"Trend (slope={trend['slope']:.4f})", linewidth=2)
plt.xlabel('Time (years)')
plt.ylabel('NDBI')
plt.title('Urban Growth Trend Analysis')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Change Point Detection

In [ ]:
# Example with change point
time_series_with_change = np.array([0.1, 0.11, 0.12, 0.11, 0.25, 0.27, 0.28, 0.29])
timestamps_change = np.arange(len(time_series_with_change))

# Detect change points
change_points = processor.detect_change_points(time_series_with_change, method='cusum')
print(f"Detected change points at indices: {change_points}")

# Visualize
plt.figure(figsize=(10, 6))
plt.plot(timestamps_change, time_series_with_change, 'o-', linewidth=2)
for cp in change_points:
    plt.axvline(x=cp, color='r', linestyle='--', alpha=0.5, label='Change Point')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Change Point Detection')
plt.grid(True, alpha=0.3)
plt.show()

## 4. Feature Extraction

In [ ]:
# Extract features from time series
features = processor.extract_features(time_series, timestamps)

print("Extracted Features:")
for key, value in features.items():
    print(f"  {key}: {value:.4f}")

## 5. Model Training

In [ ]:
# Create synthetic dataset for demonstration
print("Creating synthetic dataset...")
df, labels = create_synthetic_dataset(n_samples=1000)

print(f"Dataset shape: {df.shape}")
print(f"Features: {df.columns.tolist()}")
print(f"\nLabel distribution:")
print(f"  Shrink (0): {(labels == 0).sum()}")
print(f"  Stable (1): {(labels == 1).sum()}")
print(f"  Grow (2): {(labels == 2).sum()}")

In [ ]:
# Initialize predictor
predictor = UrbanGrowthPredictor(model_type='gradient_boosting')

# Define features to use
feature_columns = ['ndvi_trend', 'ndbi_trend', 'building_density', 
                  'road_density', 'mobility_index']

# Prepare features
X, feature_names = predictor.prepare_features(df, feature_columns)

# Train model
print("\nTraining model...")
results = predictor.train(X, labels, feature_names=feature_names,
                         n_estimators=100, max_depth=6, learning_rate=0.1)

In [ ]:
# Visualize confusion matrix
plot_confusion_matrix(results['confusion_matrix'], predictor.class_names)

## 6. Making Predictions

In [ ]:
# Create new data for prediction
test_df, _ = create_synthetic_dataset(n_samples=100)
X_test, _ = predictor.prepare_features(test_df, feature_columns)

# Make predictions
predictions = predictor.predict(X_test)
probabilities = predictor.predict_proba(X_test)

# Display summary
print("Prediction Summary:")
for i, class_name in enumerate(predictor.class_names):
    count = (predictions == i).sum()
    percentage = count / len(predictions) * 100
    print(f"  {class_name.capitalize()}: {count} tiles ({percentage:.1f}%)")

In [ ]:
# Visualize prediction distribution
plt.figure(figsize=(10, 6))
prediction_counts = []
for i in range(len(predictor.class_names)):
    prediction_counts.append((predictions == i).sum())

plt.bar(predictor.class_names, prediction_counts, color=['red', 'gray', 'green'])
plt.xlabel('Prediction Class')
plt.ylabel('Number of Tiles')
plt.title('Distribution of Urban Growth Predictions')
plt.show()

## 7. Conclusion

This notebook demonstrated:
- Signal processing techniques for temporal analysis
- Trend detection in time series
- Change point detection
- Feature extraction from spatio-temporal signals
- Training a machine learning model for urban growth prediction
- Making predictions on new data

For real-world applications, you would:
1. Acquire actual Sentinel-2 imagery from Google Earth Engine
2. Download OpenStreetMap data for the region
3. Integrate mobility data
4. Process the data through the full pipeline
5. Generate spatial predictions and visualizations